# Оцениваем регрессии на данных HS0 в спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные

### Логично, что в тоннах значимого снижения нет, потому что самые затронутые товары не самые тяжёлые...

In [4]:
library(arrow)
library(stargazer)
library(tidyverse)
library(plotly)
library(fixest)
signif.code = signif.code = c("***" = 0.001, "**" = 0.01, "*" = 0.05, "." = 0.10)

## Сырые данные

Под каждой регрессией вывожу summary с кластеризацией ошибок. В конце блока сравниваю робастность результата по используемым ошибкам.

In [5]:
# Скачиваю данные и переназываю переменные
df = read_parquet('../../data/processed_data/trade_hs0_x_covid.parquet.gzip')
# Переменная pair нужна, чтобы считать НАС стандартные ошибки
df = df %>% mutate(pair = paste(i, j))

df$deaths_i = df$deaths
df$deaths_j = df$`deaths:r_j`

df$median_age_i = df$median_age
df$median_age_j = df$`median_age:r_j`

df$average_surface_temperature_i = df$average_surface_temperature
df$average_surface_temperature_j = df$`average_surface_temperature:r_j`

df$C1M_School_closing_i = df$C1M_School_closing
df$C1M_School_closing_j = df$`C1M_School_closing:r_j`

df$population_density_i = df$population_density
df$population_density_j = df$`population_density:r_j`

df$share_1_members_i = df$share_1_members
df$share_1_members_j = df$`share_1_members:r_j`

df$H6M_Facial_Coverings_i = df$H6M_Facial_Coverings
df$H6M_Facial_Coverings_j = df$`H6M_Facial_Coverings:r_j`

df$C8EV_International_travel_controls_i = df$C8EV_International_travel_controls
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,v,q,v19,q19,country,t:r_i,cases,⋯,C1M_School_closing_i,C1M_School_closing_j,population_density_i,population_density_j,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021,LUX,LBY,968.018,210.173,501.858,127.012,LUX,2021,241.1512329,⋯,1.183562,1.49315071,248.620682,4.055137,33.34,NA,2.0082192,2.827397,2.534247,2.520548
2021,RUS,SEN,1307165.125,3131615.500,789625.305,2232100.632,RUS,2021,138.1677534,⋯,1.290411,0.07671233,8.838488,89.445110,25.69,6.92,2.3150685,3.402740,2.610959,2.561644
2020,SOM,KOR,3136.611,2045.876,5608.268,4002.770,SOM,2020,0.7276796,⋯,1.584700,1.98087430,26.542530,531.109006,1.53,31.75,0.7622951,1.653005,2.101093,2.415301
2020,TUR,CRI,57792.285,98850.453,60858.013,86163.873,TUR,2020,67.6935912,⋯,2.153005,2.40437150,108.343854,98.596161,6.37,12.07,2.2732241,1.218579,2.040984,2.322404
2021,KNA,BIH,0.520,0.008,0.460,0.005,KNA,2021,164.5090411,⋯,NA,1.00000000,179.857692,63.377090,NA,11.50,NA,2.476712,NA,1.000000
2022,AZE,MLT,22060.938,39007.156,12476.490,33493.400,AZE,2022,56.3560822,⋯,2.000000,0.27945206,122.707272,1660.971875,6.77,22.64,0.6575342,2.323288,3.000000,1.263014


In [6]:
# Делаю pulled OLS ради интереса
m0 = feols(log(q/q19) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df)
m0 %>% summary(cluster = ~ i+j)

NOTE: 13,943 observations removed because of NA and infinite values (LHS: 625, RHS: 13,568).



OLS estimation, Dep. Var.: log(q/q19)
Observations: 56,364
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
(Intercept)                           0.032537   0.025994  1.251718 0.212353
log(deaths_i)                         0.003188   0.003935  0.810295 0.418877
log(deaths_j)                        -0.004522   0.004911 -0.920683 0.358490
C8EV_International_travel_controls_i  0.000576   0.010583  0.054420 0.956663
C8EV_International_travel_controls_j -0.026123   0.010709 -2.439309 0.015719
                                        
(Intercept)                             
log(deaths_i)                           
log(deaths_j)                           
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.16484   Adj. R2: 4.346e-4

In [7]:
# FE (т.е. оценка на панельных данных с использованием внутригрупповой дисперсии)
m1 = feols(log(q/q19) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j| i + j + year,
 data = df)
m1 %>% summary(cluster = ~ i+j)

NOTE: 13,943 observations removed because of NA and infinite values (LHS: 625, RHS: 13,568).



OLS estimation, Dep. Var.: log(q/q19)
Observations: 56,364
Fixed-effects: i: 175,  j: 175,  year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
log(deaths_i)                        -0.005611   0.005950 -0.943014 0.346981
log(deaths_j)                        -0.002078   0.005677 -0.366056 0.714768
C8EV_International_travel_controls_i -0.018008   0.017993 -1.000828 0.318300
C8EV_International_travel_controls_j -0.030099   0.015274 -1.970636 0.050352
                                        
log(deaths_i)                           
log(deaths_j)                           
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.14605     Adj. R2: 0.026372
                Within R2: 1.511e-4

In [8]:
# Добавляю гетерогенность эффектов во времени
m2 = feols(log(q/q19) ~ year/log(deaths_i) + year/log(deaths_j) +
year/C8EV_International_travel_controls_i + year/C8EV_International_travel_controls_j|
i + j ,
data = df)
m2 %>% summary(cluster = ~ i+j)

NOTE: 13,943 observations removed because of NA and infinite values (LHS: 625, RHS: 13,568).



OLS estimation, Dep. Var.: log(q/q19)
Observations: 56,364
Fixed-effects: i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                               Estimate Std. Error   t value
year2021                                      -0.077166   0.111634 -0.691247
year2022                                      -0.098782   0.129504 -0.762775
year2020:log(deaths_i)                         0.000018   0.006617  0.002784
year2021:log(deaths_i)                        -0.000286   0.007938 -0.036045
year2022:log(deaths_i)                        -0.015095   0.008131 -1.856449
year2020:log(deaths_j)                        -0.007398   0.007293 -1.014345
year2021:log(deaths_j)                         0.002447   0.008267  0.295996
year2022:log(deaths_j)                         0.000223   0.007206  0.030998
year2020:C8EV_International_travel_controls_i -0.038958   0.035296 -1.103752
year2021:C8EV_International_travel_controls_i -0.016029   0.017505 -0.915649
year2022:C8EV_International

In [ ]:
etable(m0, m1, m2, vcov = ~i+j, signif.code=signif.code)

,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,Constant,0.0325 (0.0260),,
4,log(deaths_i),0.0032 (0.0039),-0.0056 (0.0060),
5,log(deaths_j),-0.0045 (0.0049),-0.0021 (0.0057),
6,C8EV_International_travel_controls_i,0.0006 (0.0106),-0.0180 (0.0180),
7,C8EV_International_travel_controls_j,-0.0261* (0.0107),-0.0301. (0.0153),
8,year2021,,,-0.0772 (0.1116)
9,year2022,,,-0.0988 (0.1295)


In [ ]:
etable(m0, m1, m2, vcov = vcov_NW(unit = 'pair', time = 'year'), signif.code=signif.code)

,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,Constant,0.0325 (0.0157),,
4,log(deaths_i),0.0032 (0.0035),-0.0056 (0.0065),
5,log(deaths_j),-0.0045 (0.0033),-0.0021 (0.0067),
6,C8EV_International_travel_controls_i,0.0006 (0.0082),-0.0180 (0.0141),
7,C8EV_International_travel_controls_j,-0.0261. (0.0083),-0.0301 (0.0138),
8,year2021,,,-0.0772 (0.0882)
9,year2022,,,-0.0988 (0.0913)


In [ ]:
etable(m0, m1, m2, vcov = ~i+j+year, signif.code=signif.code)

Variance contained negative values in the diagonal and was 'fixed' (a la Cameron, Gelbach & Miller 2011).



,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,Constant,0.0325** (0.0032),,
4,log(deaths_i),0.0032 (0.0054),-0.0056 (0.0051),
5,log(deaths_j),-0.0045 (0.0043),-0.0021 (0.0049),
6,C8EV_International_travel_controls_i,0.0006 (0.0058),-0.0180 (0.0133),
7,C8EV_International_travel_controls_j,-0.0261. (0.0062),-0.0301 (0.0191),
8,year2021,,,-0.0772 (0.0932)
9,year2022,,,-0.0988 (0.1061)


In [12]:
# Не понятно, насколько это осмысленно, но сделал на всякий случай
etable(m0, m1, m2, vcov = ~i^year+j^year, signif.code=signif.code)

,,m0,m1,m2
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,Constant,0.0325 (0.0278),,
4,log(deaths_i),0.0032 (0.0034),-0.0056 (0.0048),
5,log(deaths_j),-0.0045 (0.0039),-0.0021 (0.0049),
6,C8EV_International_travel_controls_i,0.0006 (0.0115),-0.0180 (0.0149),
7,C8EV_International_travel_controls_j,-0.0261* (0.0123),-0.0301* (0.0124),
8,year2021,,,-0.0772 (0.0868)
9,year2022,,,-0.0988 (0.1019)


### Инструментальные переменные на панельных данных

In [13]:
# Pulled TSLS
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
m3 = feols(log(q/q19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) 
 ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
population_density + population_density_j
  ,
 data = df
 )
m3 %>% summary(vcov = ~i+j)

NOTE: 14,279 observations removed because of NA and infinite values (LHS: 625, RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(q/q19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(q/q19)
Observations: 56,028
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
(Intercept)                           0.123115   0.058461  2.105938 0.036645
fit_log(deaths_i)                    -0.021927   0.074768 -0.293263 0.769670
fit_log(deaths_j)                     0.095218   0.057095  1.667698 0.097174
C8EV_International_travel_controls_i -0.023711   0.022238 -1.066268 0.287780
C8EV_International_travel_controls_j -0.011135   0.025447 -0.437573 0.662239
                                        
(Intercept)                          *  
fit_log(deaths_i)                       
fit_log(deaths_j)                    .  
C8EV_International_travel_controls_i    
C8EV_International_trave

In [14]:
# Только FE года (инструменты эндогенные)
m4 = feols(log(q/q19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year|
 log(deaths_i) + log(deaths_j) ~ 
 median_age_i + median_age_j 
+
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
  population_density + population_density_j
 ,
 data = df
 )
m4 %>% summary(cluster = ~ i+j)

NOTE: 14,279 observations removed because of NA and infinite values (LHS: 625, RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(q/q19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : median_age_i, median_age_j, H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, population_density, population_density_j
Second stage: Dep. Var.: log(q/q19)
Observations: 56,028
Fixed-effects: year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i)                     0.000274   0.007428  0.036838  0.97066 
fit_log(deaths_j)                    -0.010887   0.009492 -1.147029  0.25294 
C8EV_International_travel_controls_i  0.005780   0.014937  0.386917  0.69929 
C8EV_International_travel_controls_j -0.022106   0.013994 -1.579691  0.11599 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.16345     Adj. R2: 0.001006
                Within R2: 2.542e-4
F-test (1st stage), log(deaths_i): stat = 7,438.6     , p < 2.2e-16 , on 6 and 56,019 DoF.
F-test (1st stage), 

In [ ]:

feols(log(q/q19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j

 ,
 data = df
 ) %>% summary(cluster = ~ i+j)

NOTE: 14,279 observations removed because of NA and infinite values (LHS: 625, RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(q/q19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(q/q19)
Observations: 56,028
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(deaths_i)                    -0.020314   0.024631 -0.824739 0.410649
fit_log(deaths_j)                    -0.012250   0.020705 -0.591662 0.554845
C8EV_International_travel_controls_i -0.022934   0.018475 -1.241405 0.216127
C8EV_International_travel_controls_j -0.032554   0.017024 -1.912287 0.057482
                                        
fit_log(deaths_i)                       
fit_log(deaths_j)                       
C8EV_International_travel_controls_i    
C8EV_International_travel_c

In [16]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 
# Добавить больше инструментов, которые меняются во времени. Мб плотность населения или медианный возраст брать не за 2019 год а в динамике, 
# чтобы использовать их как инструменты в этой спецификации.
m5 = feols(log(q/q19) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
m5 %>% summary(cluster = ~ i+j)

NOTE: 14,279 observations removed because of NA and infinite values (LHS: 625, RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(q/q19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(q/q19)
Observations: 56,028
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(deaths_i)                    -0.023478   0.025714 -0.913025  0.36249
fit_log(deaths_j)                    -0.006637   0.021853 -0.303715  0.76171
C8EV_International_travel_controls_i -0.024346   0.018960 -1.284121  0.20081
C8EV_International_travel_controls_j -0.030037   0.017049 -1.761795  0.07986
                                        
fit_log(deaths_i)                       
fit_log(deaths_j)                       
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j .  
---
Signif. codes:  0 '***' 0.001 

In [17]:
etable(m3, m4, m5, vcov = ~i+j, signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,Constant,0.1231* (0.0585),,
4,log(deaths_i),-0.0219 (0.0748),0.0003 (0.0074),-0.0235 (0.0257)
5,log(deaths_j),0.0952. (0.0571),-0.0109 (0.0095),-0.0066 (0.0219)
6,C8EV_International_travel_controls_i,-0.0237 (0.0222),0.0058 (0.0149),-0.0244 (0.0190)
7,C8EV_International_travel_controls_j,-0.0111 (0.0254),-0.0221 (0.0140),-0.0300. (0.0170)
8,Fixed-Effects:,----------------,----------------,-----------------
9,year,No,Yes,Yes


In [18]:
etable(m3, m4, m5, vcov = vcov_NW(unit = 'pair', time = 'year'), signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,Constant,0.1231. (0.0318),,
4,log(deaths_i),-0.0219 (0.0286),0.0003 (0.0056),-0.0235 (0.0244)
5,log(deaths_j),0.0952. (0.0295),-0.0109 (0.0051),-0.0066 (0.0222)
6,C8EV_International_travel_controls_i,-0.0237 (0.0118),0.0058 (0.0110),-0.0244 (0.0178)
7,C8EV_International_travel_controls_j,-0.0111 (0.0118),-0.0221 (0.0107),-0.0300 (0.0163)
8,Fixed-Effects:,----------------,----------------,----------------
9,year,No,Yes,Yes


In [19]:
    # Всё ещё странно использовать кластеризацию по году... 
    # etable(m3, m4, vcov = function(x) vcovHC(x, method="arellano", type="HC1"), signif.code = signif.code) Не сработало
etable(m3, m4, m5, vcov = ~i+j+year, signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,Constant,0.1231 (0.0957),,
4,log(deaths_i),-0.0219 (0.0946),0.0003 (0.0080),-0.0235 (0.0230)
5,log(deaths_j),0.0952 (0.0407),-0.0109 (0.0094),-0.0066 (0.0181)
6,C8EV_International_travel_controls_i,-0.0237 (0.0223),0.0058 (0.0084),-0.0244 (0.0132)
7,C8EV_International_travel_controls_j,-0.0111 (0.0201),-0.0221 (0.0128),-0.0300 (0.0187)
8,Fixed-Effects:,----------------,----------------,----------------
9,year,No,Yes,Yes


In [20]:
# Так вроде не очень корректно делать, если мы верим в возможную автокорреляцию
etable(m3, m4, m5, vcov = ~i^year+j^year, signif.code = signif.code)

,,m3,m4,m5
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,Constant,0.1231* (0.0573),,
4,log(deaths_i),-0.0219 (0.0541),0.0003 (0.0054),-0.0235 (0.0214)
5,log(deaths_j),0.0952* (0.0463),-0.0109. (0.0063),-0.0066 (0.0181)
6,C8EV_International_travel_controls_i,-0.0237 (0.0196),0.0058 (0.0142),-0.0244 (0.0160)
7,C8EV_International_travel_controls_j,-0.0111 (0.0215),-0.0221 (0.0141),-0.0300* (0.0140)
8,Fixed-Effects:,----------------,-----------------,-----------------
9,year,No,Yes,Yes


In [21]:
# С помощью fixest::demean можно отрисовать scatterplot, который обосновывает необходимость использовать фиксированные эффекты
# и релевантность инструментов

## Три пространственные регрессии (как в курсовой)

Но в такую спецификацию нельзя добавить фиксированные эффекты, поэтому почти наверное без инструментов будет смещение из-за пропущенных переменных

In [22]:
# Без инструментов.
m_2020 = feols(log(q/q19) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j ,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(q/q19) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(q/q19) ~  1+log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)

NOTE: 5,636 observations removed because of NA and infinite values (LHS: 190, RHS: 5,531).

NOTE: 4,255 observations removed because of NA and infinite values (LHS: 207, RHS: 4,126).

NOTE: 4,052 observations removed because of NA and infinite values (LHS: 228, RHS: 3,911).



OLS estimation, Dep. Var.: log(q/q19)
Observations: 18,286
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value Pr(>|t|)
(Intercept)                          -0.067531   0.053271 -1.267692 0.204924
log(deaths_i)                         0.004973   0.003730  1.333213 0.182479
log(deaths_j)                        -0.010661   0.003802 -2.804230 0.005049
C8EV_International_travel_controls_i  0.009176   0.015316  0.599070 0.549134
C8EV_International_travel_controls_j -0.010404   0.014605 -0.712335 0.476266
                                        
(Intercept)                             
log(deaths_i)                           
log(deaths_j)                        ** 
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.01185   Adj. R2: 3.98e-4

OLS estimation, Dep. Var.: log(q/q19)
Observations: 19,303
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value Pr(>|t|)
(Intercept)                           0.008210   0.039045  0.210278 0.833453
log(deaths_i)                         0.000752   0.005054  0.148799 0.881713
log(deaths_j)                        -0.009614   0.004845 -1.984162 0.047252
C8EV_International_travel_controls_i  0.011073   0.011366  0.974212 0.329963
C8EV_International_travel_controls_j -0.011522   0.011430 -1.008079 0.313429
                                        
(Intercept)                             
log(deaths_i)                           
log(deaths_j)                        *  
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.17955   Adj. R2: 1.147e-4

OLS estimation, Dep. Var.: log(q/q19)
Observations: 18,775
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error  t value  Pr(>|t|)
(Intercept)                           0.059646   0.023965  2.48892 0.0128218
log(deaths_i)                        -0.007681   0.005576 -1.37744 0.1683928
log(deaths_j)                        -0.005446   0.004909 -1.10926 0.2673326
C8EV_International_travel_controls_i -0.024098   0.022238 -1.08365 0.2785340
C8EV_International_travel_controls_j -0.053444   0.020304 -2.63222 0.0084899
                                        
(Intercept)                          *  
log(deaths_i)                           
log(deaths_j)                           
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.28152   Adj. R2: 3.109e-4

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,Constant,-0.0675 (0.0533),0.0082 (0.0390),0.0597* (0.0240)
4,log(deaths_i),0.0050 (0.0037),0.0008 (0.0051),-0.0077 (0.0056)
5,log(deaths_j),-0.0107** (0.0038),-0.0096* (0.0048),-0.0054 (0.0049)
6,C8EV_International_travel_controls_i,0.0092 (0.0153),0.0111 (0.0114),-0.0241 (0.0222)
7,C8EV_International_travel_controls_j,-0.0104 (0.0146),-0.0115 (0.0114),-0.0534** (0.0203)
8,____________________________________,__________________,_________________,__________________
9,S.E. type,Heteroskedas.-rob.,Heteroskeda.-rob.,Heteroskedas.-rob.


In [23]:
# С инструментами оценки коэффициентов всё равно не похожи на панельные данные (пространственные смещённые, т.к должно быть много unobservables)
# Использую ошибки hc1
# Подумать про эндогенность пропусков. Мб пропуски в инструментальных переменных можно смело заполнять?
# Почему экзогенные в панельной спецификации инструменты эндогенны в пространственной?
m_2020 = feols(log(q/q19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2020')
 )
m_2021 = feols(log(q/q19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+ median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2021')
 )
m_2022 = feols(log(q/q19) ~  1+
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j |
 log(deaths_i) + log(deaths_j) ~ 
 C1M_School_closing_i+C1M_School_closing_j+ median_age_i + median_age_j + population_density_i + population_density_j,
 data = df %>% filter(year == '2022')
 )
summary(m_2020, vcov = 'hc1') 
summary(m_2021, vcov = 'hc1') 
summary(m_2022, vcov = 'hc1') 
etable(m_2020, m_2021, m_2022, vcov = 'hc1', signif.code = signif.code)
etable(m_2020, m_2021, m_2022, vcov = ~i+j, signif.code = signif.code)

NOTE: 5,636 observations removed because of NA and infinite values (LHS: 190, RHS: 3,419, IV: 3,381/3,419).

NOTE: 4,255 observations removed because of NA and infinite values (LHS: 207, RHS: 3,290, IV: 1,557/3,290).

NOTE: 4,388 observations removed because of NA and infinite values (LHS: 228, RHS: 3,093, IV: 1,129/3,952).



TSLS estimation - Dep. Var.: log(q/q19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(q/q19)
Observations: 18,286
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value   Pr(>|t|)
(Intercept)                          -0.071379   0.053297 -1.339259 0.18050291
fit_log(deaths_i)                     0.007028   0.005975  1.176224 0.23952053
fit_log(deaths_j)                    -0.019235   0.005608 -3.430114 0.00060466
C8EV_International_travel_controls_i  0.009653   0.015412  0.626360 0.53108647
C8EV_International_travel_controls_j -0.012947   0.014625 -0.885291 0.37601151
                                        
(Intercept)                             
fit_log(deaths_i)                       
fit_log(deaths_j)                    ***
C8EV_International_

TSLS estimation - Dep. Var.: log(q/q19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(q/q19)
Observations: 19,303
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error   t value Pr(>|t|) 
(Intercept)                           0.008739   0.038886  0.224742  0.82218 
fit_log(deaths_i)                    -0.001592   0.008606 -0.185010  0.85322 
fit_log(deaths_j)                    -0.006384   0.007701 -0.828958  0.40714 
C8EV_International_travel_controls_i  0.011321   0.011364  0.996216  0.31916 
C8EV_International_travel_controls_j -0.011820   0.011419 -1.035080  0.30064 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.17957   Adj. R2: 7.939e-5
F-test (1st stage), log(deaths_i): stat = 2,073.9     , p < 2.2e-16 , on 6 and 19,294 Do

TSLS estimation - Dep. Var.: log(q/q19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(q/q19)
Observations: 18,439
Standard-errors: Heteroskedasticity-robust 
                                      Estimate Std. Error  t value  Pr(>|t|)
(Intercept)                           0.055323   0.024461  2.26171 0.0237267
fit_log(deaths_i)                    -0.011974   0.006418 -1.86577 0.0620889
fit_log(deaths_j)                    -0.011238   0.005912 -1.90078 0.0573470
C8EV_International_travel_controls_i -0.027037   0.022719 -1.19005 0.2340412
C8EV_International_travel_controls_j -0.062926   0.020877 -3.01419 0.0025802
                                        
(Intercept)                          *  
fit_log(deaths_i)                    .  
fit_log(deaths_j)                    .  
C8EV_International_travel_contr

,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,Constant,-0.0714 (0.0533),0.0087 (0.0389),0.0553* (0.0245)
4,log(deaths_i),0.0070 (0.0060),-0.0016 (0.0086),-0.0120. (0.0064)
5,log(deaths_j),-0.0192*** (0.0056),-0.0064 (0.0077),-0.0112. (0.0059)
6,C8EV_International_travel_controls_i,0.0097 (0.0154),0.0113 (0.0114),-0.0270 (0.0227)
7,C8EV_International_travel_controls_j,-0.0129 (0.0146),-0.0118 (0.0114),-0.0629** (0.0209)
8,____________________________________,___________________,________________,__________________
9,S.E. type,Heteroskedast.-rob.,Heterosked.-rob.,Heteroskedas.-rob.


,,m_2020,m_2021,m_2022
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(q/q19),log(q/q19),log(q/q19)
2,,,,
3,Constant,-0.0714 (0.0748),0.0087 (0.0539),0.0553 (0.0536)
4,log(deaths_i),0.0070 (0.0064),-0.0016 (0.0094),-0.0120 (0.0106)
5,log(deaths_j),-0.0192* (0.0082),-0.0064 (0.0117),-0.0112 (0.0105)
6,C8EV_International_travel_controls_i,0.0097 (0.0199),0.0113 (0.0165),-0.0270 (0.0517)
7,C8EV_International_travel_controls_j,-0.0129 (0.0222),-0.0118 (0.0170),-0.0629 (0.0431)
8,____________________________________,_________________,________________,________________
9,S.E.: Clustered,by: i & j,by: i & j,by: i & j
